In [19]:
import spacy
nlp = spacy.load("de_dep_news_trf") #insert the name of the model you want to use (preload if necessary)

In [20]:
import functions #functions for normalization and sentence splitting

In [21]:
import pandas as pd

In [29]:
#using the deTenTen23 or OneMillionPostsCorpus corpus as an example
df = pd.read_excel('OMPC_postswith_gehoeren.xlsx') #insert the name of your excel file (or csv file)

In [30]:
# adding id numbers
df['sentence_id'] = df.reset_index().index +1
print(df.count())
print(df.head())

ID_Post        17508
CreatedAt      17508
Headline        6267
Sentence       17508
sentence_id    17508
dtype: int64
   ID_Post                CreatedAt  \
0       40  2009-04-05 10:58:51.037   
1      238  2014-08-13 09:55:10.240   
2      341  2014-08-13 13:22:25.483   
3      505  2014-08-14 17:02:57.707   
4      530  2014-08-17 14:41:13.097   

                                            Headline  \
0       derStandard ist weit mehr als nur "Standard"   
1                       wie schon unten erwähnt ....   
2                                                NaN   
3  mein Kind ist Linkshänder und benutzt das Best...   
4                                                NaN   

                                            Sentence  sentence_id  
0  Aber es gehört zum Standard, täglich diese Zei...            1  
1  saucenschöpfer mit schnabel und dazu gehören a...            2  
2  ), alles links. Lechts und rinks zu unterschei...            3  
3  In der VS hben sie einmal gekocht u

In [31]:
# Normalize the text in the dataframe
df['text'] = df['Sentence'].apply(functions.normalize_text)
df = df.drop(columns=['Sentence'])
df.head()

,ID_Post,CreatedAt,Headline,sentence_id,text
0,40,2009-04-05 10:58:51.037,"derStandard ist weit mehr als nur ""Standard""",1,"Aber es gehört zum Standard, täglich diese Zei..."
1,238,2014-08-13 09:55:10.240,wie schon unten erwähnt ....,2,saucenschöpfer mit schnabel und dazu gehören a...
2,341,2014-08-13 13:22:25.483,NaN,3,"), alles links. Lechts und rinks zu unterschei..."
3,505,2014-08-14 17:02:57.707,mein Kind ist Linkshänder und benutzt das Best...,4,In der VS hben sie einmal gekocht und mein Kin...
4,530,2014-08-17 14:41:13.097,NaN,5,.... was rechtshändern vermutlich unmöglich is...


In [32]:
# Define the delimiters
delimiters = [r'[^\w\s]', r'\boder\b', r'\bund\b']

# Apply the function to the dataframe
df = functions.split_and_explode(df, 'text', delimiters)
print(df.head())
print(df.count())

   ID_Post                CreatedAt  \
0       40  2009-04-05 10:58:51.037   
0       40  2009-04-05 10:58:51.037   
0       40  2009-04-05 10:58:51.037   
0       40  2009-04-05 10:58:51.037   
0       40  2009-04-05 10:58:51.037   

                                       Headline  sentence_id  \
0  derStandard ist weit mehr als nur "Standard"            1   
0  derStandard ist weit mehr als nur "Standard"            1   
0  derStandard ist weit mehr als nur "Standard"            1   
0  derStandard ist weit mehr als nur "Standard"            1   
0  derStandard ist weit mehr als nur "Standard"            1   

                                                text  \
0  Aber es gehört zum Standard, täglich diese Zei...   
0  Aber es gehört zum Standard, täglich diese Zei...   
0  Aber es gehört zum Standard, täglich diese Zei...   
0  Aber es gehört zum Standard, täglich diese Zei...   
0  Aber es gehört zum Standard, täglich diese Zei...   

                                        tex

In [33]:
df = df.drop(columns=['text_split_0', 'text_split_0_split_1']) #adjust if you have more  or less delimiters

In [34]:
#pre-filtering
df['gehören_yes']=df['final_split'].str.contains(r'geh(ö|oe|o)r*') #adjust the regex to your phenomenon
df = df[df['gehören_yes']]

print(df.count)

<bound method DataFrame.count of        ID_Post                CreatedAt  \
0           40  2009-04-05 10:58:51.037   
1          238  2014-08-13 09:55:10.240   
2          341  2014-08-13 13:22:25.483   
3          505  2014-08-14 17:02:57.707   
4          530  2014-08-17 14:41:13.097   
...        ...                      ...   
17504  1011670  2016-06-01 17:37:39.323   
17505  1011722  2016-06-02 08:46:27.943   
17506  1011742  2016-06-03 02:23:48.523   
17506  1011742  2016-06-03 02:23:48.523   
17507  1011743  2016-06-03 02:25:20.677   

                                                Headline  sentence_id  \
0           derStandard ist weit mehr als nur "Standard"            1   
1                           wie schon unten erwähnt ....            2   
2                                                    NaN            3   
3      mein Kind ist Linkshänder und benutzt das Best...            4   
4                                                    NaN            5   
...         

C:\Users\matte\AppData\Local\Temp\ipykernel_24896\3484620260.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['gehören_yes']=df['final_split'].str.contains(r'geh(ö|oe|o)r*') #adjust the regex to your phenomenon


In [35]:
df = df.drop_duplicates(subset='sentence_id')

In [36]:
#filter sentences for the target
target_lemma = 'gehören'        #adjust the lemma to your target phenomenon
target_tag = 'VVPP'             #adjust the tag to your target phenomenon

In [37]:
# extract lemma
def extract_lemma(text):
    doc = nlp(text)
    lemma = [token.text for token in doc if token.lemma_ == target_lemma]
    return ', '.join(lemma)

# Apply the function to the column and create a new column with the extracted verbs
df['gehören'] = df['final_split'].apply(extract_lemma)

In [38]:
# Function to extract the closest VVPP tagged verb to gehören
def extract_closest_tag(text):
    doc = nlp(text)
    # Find the lemma token
    lemma_token = None
    for token in doc:
        if token.lemma_ == target_lemma:
            lemma_token = token
            break
    
    if not lemma_token:
        return None
    
    # Find closest VVPP
    closest_vvpp = None
    min_distance = float('inf')
    
    for token in doc:
        if token.tag_ == target_tag and token.lemma_ != target_lemma:
            # Check if there are any punctuation between the tokens
            start_idx = min(lemma_token.i, token.i)
            end_idx = max(lemma_token.i, token.i)
            tokens_between = doc[start_idx:end_idx]
            if not any(t.is_punct for t in tokens_between):
                distance = abs(token.i - lemma_token.i)
                if distance < min_distance:
                    min_distance = distance
                    closest_vvpp = token.text
    
    return closest_vvpp

df['VVPP'] = df['final_split'].apply(extract_closest_tag)

In [39]:
#nan values the df
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.head()

,ID_Post,CreatedAt,Headline,sentence_id,text,final_split,gehören_yes,gehören,VVPP
0,40,2009-04-05 10:58:51.037,"derStandard ist weit mehr als nur ""Standard""",1,"Aber es gehört zum Standard, täglich diese Zei...",Aber es gehört zum Standard,True,gehört,None
1,238,2014-08-13 09:55:10.240,wie schon unten erwähnt ....,2,saucenschöpfer mit schnabel und dazu gehören a...,dazu gehören auch noch messer vom fischbesteck,True,gehören,None
2,341,2014-08-13 13:22:25.483,NaN,3,"), alles links. Lechts und rinks zu unterschei...",rinks zu unterscheiden bedarf einer gehörigen...,True,NaN,None
3,505,2014-08-14 17:02:57.707,mein Kind ist Linkshänder und benutzt das Best...,4,In der VS hben sie einmal gekocht und mein Kin...,dass der Löffel eigentlich rechts neben den T...,True,gehört,None
4,530,2014-08-17 14:41:13.097,NaN,5,.... was rechtshändern vermutlich unmöglich is...,vor nicht allzu langer zeit habe ich ein inte...,True,NaN,None


In [40]:
#remove the rows with NaN
df.dropna(subset = ['gehören', 'VVPP'], inplace =True)
df.head()

,ID_Post,CreatedAt,Headline,sentence_id,text,final_split,gehören_yes,gehören,VVPP
5,533,2014-08-21 13:46:03.700,NaN,6,"gehört im kindesalter umgelernt , ganz einfach.",gehört im kindesalter umgelernt,True,gehört,umgelernt
38,2642,2015-03-11 15:16:04.713,NaN,39,diese praktikumsgehälter sind wirklich ein wit...,gehört abgeschafft,True,gehört,abgeschafft
53,3116,2015-03-15 01:21:54.553,NaN,54,*piketty soll es natürlich heißen. es war scho...,es gehören mal ein paar dinge klar gestellt,True,gehören,gestellt
55,3166,2015-03-16 23:02:47.570,NaN,56,das gehört ausgeschmückt und mit absätzen vers...,das gehört ausgeschmückt,True,gehört,ausgeschmückt
57,3243,2015-03-26 15:35:06.027,NaN,58,Konkurrenzklauseln gehören abgeschafft. Sie si...,Konkurrenzklauseln gehören abgeschafft,True,gehören,abgeschafft


In [41]:
df.count()

ID_Post        2690
CreatedAt      2690
Headline       1019
sentence_id    2690
text           2690
final_split    2690
gehören_yes    2690
gehören        2690
VVPP           2690
dtype: int64

In [42]:
print(df)
df.to_excel('gehören_passive_filtered_OMPC.xlsx', index=False)

       ID_Post                CreatedAt  \
5          533  2014-08-21 13:46:03.700   
38        2642  2015-03-11 15:16:04.713   
53        3116  2015-03-15 01:21:54.553   
55        3166  2015-03-16 23:02:47.570   
57        3243  2015-03-26 15:35:06.027   
...        ...                      ...   
17471  1010386  2016-06-01 07:58:53.327   
17478  1010783  2016-06-01 00:04:20.483   
17480  1010925  2016-06-01 15:14:48.713   
17482  1011020  2016-05-31 18:50:56.873   
17487  1011070  2016-05-31 20:35:05.293   

                                                Headline  sentence_id  \
5                                                    NaN            6   
38                                                   NaN           39   
53                                                   NaN           54   
55                                                   NaN           56   
57                                                   NaN           58   
...                                          

In [30]:
df['Domain'].value_counts()

Domain
de      85
at      21
com     19
net      9
ch       3
org      2
eu       2
info     2
blog     1
news     1
Name: count, dtype: int64

In [31]:
df['VVPP'].value_counts()

VVPP
verboten        9
gesehen         5
abgeschafft     4
diskutiert      3
bestraft        3
               ..
entmachtet      1
verbracht       1
erlegt          1
veranstaltet    1
weggesperrt     1
Name: count, Length: 110, dtype: int64